In [1]:
import tensorflow as tf
import pandas as pd
from utilities import *
import keras
from pandas.api.types import is_string_dtype, is_numeric_dtype
from keras.models import Model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform, RandomNormal
from keras import backend as K
from keras import losses

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('../data/train.csv',index_col=0)

In [3]:
data = data[['sentiment_sum',
 'count',
 'sentiment_mean',
 'weighted_sentiment_sum',
 'weighted_sentiment_mean',
 'mean_price_change',
 'preClosePrice',
 'actPreClosePrice',
 'openPrice',
 'negMarketValue',
 'marketValue',
 'PE',
 'PE1',
 'PB']]

In [4]:
data.head(2)

,sentiment_sum,count,sentiment_mean,weighted_sentiment_sum,weighted_sentiment_mean,mean_price_change,preClosePrice,actPreClosePrice,openPrice,negMarketValue,marketValue,PE,PE1,PB
0,2,2,1.0,0.138345,0.069172,-0.004720,9.04,9.04,9.0,10571320662,10627250487,5.2821,7.3733,2.5812
1,0,1,0.0,0.000000,0.000000,0.017937,9.93,9.93,10.0,11502414426,11563270401,5.8022,8.0228,2.8086


In [5]:
data_train = data.sample(4065)

In [6]:
data_test = data.drop(data_train.index)

In [7]:
df, y, nas, mapper = proc_df(data_train, 'mean_price_change', do_scale=True)
# log+1 transformation
yl = np.log(y+1)

In [8]:
df_test, y_test, nas, mapper = proc_df(data_test, 'mean_price_change', do_scale=True,
                                  mapper=mapper, na_dict=nas)

In [9]:
df.shape,yl.shape

((4065, 13), (4065,))

In [10]:
y_test_l = np.log(y_test+1)

## Model

In [11]:
def inv_y(a): return np.exp(a)-1

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())
max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [12]:
def dot__(X,W,b):
    return K.dot(X,W)+b

In [13]:
n_x = df.shape[1]

In [14]:
input_tensor = Input((n_x,))
X = input_tensor
X2 = BatchNormalization()(X)
W = RandomNormal(mean=0.0, stddev=0.05, seed=None)([1000,500])
b = RandomNormal(mean=0.0, stddev=0.05, seed=None)([500,])
X = Dense(1000)(X2)
X = Lambda(lambda x, W, b: dot__(x,W,b),arguments={'W':W,'b':b})(X)
X = Activation('relu')(X)
X = Dropout(0.001)(X)
X = Dense(1000)(X)
X = Lambda(lambda x, W, b: dot__(x,W,b),arguments={'W':W,'b':b})(X)
X = Activation('relu')(X)
X = Dropout(0.01)(X)
X = Activation('sigmoid')(X)
X = Lambda(lambda x: x*(y_range[1]-y_range[0]))(X)
X = Lambda(lambda x: x+y_range[0])(X)
X = Dense(64)(X)
X = Dense(1)(X)
Y = X

In [15]:
model = Model(inputs=[input_tensor],outputs=[Y])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 13)                52        
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              14000     
_________________________________________________________________
lambda_1 (Lambda)            (None, 500)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              501000    
__________

In [17]:
model.compile(loss=losses.mean_squared_error, optimizer='adam',metrics=['msle'])

In [19]:
from keras.utils import plot_model
plot_model(model, to_file='../images/model.png',show_shapes=True)

In [29]:
model.fit(np.array(df), yl, epochs = 100, batch_size = 256, validation_split=0.1, shuffle=True)

Train on 3658 samples, validate on 407 samples
Epoch 1/100
3658/3658 [==============================] - 0s 40us/step - loss: 0.0016 - mean_squared_logarithmic_error: 5.0642e-04 - val_loss: 0.0034 - val_mean_squared_logarithmic_error: 4.4588e-04
Epoch 2/100
3658/3658 [==============================] - 0s 33us/step - loss: 0.0014 - mean_squared_logarithmic_error: 4.8308e-04 - val_loss: 0.0036 - val_mean_squared_logarithmic_error: 5.1873e-04
Epoch 3/100
3658/3658 [==============================] - 0s 31us/step - loss: 0.0015 - mean_squared_logarithmic_error: 4.8304e-04 - val_loss: 0.0031 - val_mean_squared_logarithmic_error: 4.4162e-04
Epoch 4/100
3658/3658 [==============================] - 0s 31us/step - loss: 0.0014 - mean_squared_logarithmic_error: 4.4667e-04 - val_loss: 0.0030 - val_mean_squared_logarithmic_error: 5.1873e-04
Epoch 5/100
3658/3658 [==============================] - 0s 30us/step - loss: 0.0013 - mean_squared_logarithmic_error: 4.8239e-04 - val_loss: 0.0029 - val_mean_s

In [30]:
model.evaluate(x=np.array(df_test), y=y_test_l, batch_size=128, steps=None)

500/500 [==============================] - 0s 28us/step


[0.0023641309500671923, 0.0004500858234241605]

In [31]:
model.save_weights('./model_final.h5')

In [35]:
model.predict(x=np.array(df_test),batch_size=128)-y_test_l

array([[ 0.00381659, -0.00061441,  0.06937043, ..., -0.00801677,
         0.0174587 , -0.0159248 ],
       [ 0.00384937, -0.00058163,  0.06940321, ..., -0.00798399,
         0.01749148, -0.01589202],
       [ 0.00538058,  0.00094958,  0.07093442, ..., -0.00645278,
         0.01902269, -0.01436081],
       ...,
       [ 0.00342734, -0.00100365,  0.06898119, ..., -0.00840602,
         0.01706946, -0.01631405],
       [ 0.00093938, -0.00349161,  0.06649323, ..., -0.01089398,
         0.0145815 , -0.01880201],
       [ 0.00284198, -0.00158902,  0.06839582, ..., -0.00899138,
         0.01648409, -0.01689941]])

In [151]:
import tushare as ts
from get_sentiment import get_sentiment
from datetime import datetime

In [130]:
stks = ts.get_stock_basics()[['name']]

In [131]:
stk_2_id = dict(zip(stks['name'].tolist(),list(stks.index)))

In [141]:
news = ts.get_latest_news(500)[['title','time']]

In [142]:
news_usable = [x for x in news['title'].tolist() if any([i for i in stks['name'].tolist() if i in x])]

In [143]:
news = news[news['title'].isin(news_usable)]

In [156]:
news.loc[:,'time'] = [datetime.strptime(i,'%m-%d %M:%S') for i in news['time'].tolist()]

In [159]:
news = news[news['time'] >= datetime(1900,8,1)]

In [160]:
news.loc[:,'stock'] = 0
news.loc[:,'ticker'] = 0
news.loc[:,'sentiment'] = 0
news.loc[:,'confidence'] = 0

In [163]:
for x in news.index:
    stk = [i for i in stks['name'].tolist() if i in news.loc[x,'title']]
    if len(stk) > 0:
        stk = stk[0]
        news.loc[x,'stock'] = stk
        news.loc[x,'ticker'] = stk_2_id[stk]
        news.loc[x,'sentiment'] = get_sentiment(news.loc[x,'title'])[0]
        news.loc[x,'confidence'] = get_sentiment(news.loc[x,'title'])[1]

C:\Users\Administrator\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\Administrator\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: Insecu

In [164]:
news

,title,time,stock,ticker,sentiment,confidence
53,当代东方复牌：现金流堪忧 并购对赌之路还能走多久,1900-08-01 00:12:02,当代东方,000673,0,0.326282
54,纳川股份耗时半年收购失利 在下一盘什么样的棋？,1900-08-01 00:11:52,纳川股份,300198,0,0.285697
60,众高管回应负面标签 华大基因否认在癌变门中有责任,1900-08-01 00:11:20,华大基因,300676,0,0.115043
83,合规时间内修正业绩预告 青龙管业董秘仍被认不勤勉,1900-08-01 00:10:05,青龙管业,002457,0,0.161499
89,华大基因碧桂园抢人 济宁市前市长梅永红是何方神圣?,1900-08-01 00:09:49,华大基因,300676,0,0.000222
102,被移送公安机关还不够 *ST华泽收到深交所关注函,1900-08-01 00:08:39,*ST华泽,000693,0,0.635426
104,基建概念爆发：成都路桥8天7涨停 游资接力围攻,1900-08-01 00:08:35,成都路桥,002628,2,0.116169
106,高升控股实控人巨债缠身掏空公司? 项目烂尾欠薪保安,1900-08-01 00:08:32,高升控股,000971,2,0.060628
107,7月104家上市公司半年报业绩变脸 牧原股份价格不佳,1900-08-01 00:08:31,牧原股份,002714,0,0.486320
108,诉讼缠身、实控人还想跑 盛运环保正经历多事之秋,1900-08-01 00:08:25,盛运环保,300090,2,0.461837
